# Reading `nomisma.org` data for CRRO

This notebook loads data for the *Coinage of the Roman Republic Online* (CRRO) from `nomisma.org`'s from a delimited text file built from nomisma's RDF source, and analyzes the character set used in the text of coin legends.  It uses version `2.0.1` of the `nomisma` library.

## Configure Jupyter notebook

Configure the notebook to find the `nomisma` library.  (You could do the same thing in other environments with `sbt` or `maven`.)

In [1]:
// 1. Add maven repository where we can find our libraries
val myBT = coursierapi.MavenRepository.of("https://dl.bintray.com/neelsmith/maven")
interp.repositories() ++= Seq(myBT)

myBT: coursierapi.MavenRepository = MavenRepository(https://dl.bintray.com/neelsmith/maven)

In [2]:
// 2. Make libraries available with `$ivy` imports:
import $ivy.`edu.holycross.shot::nomisma:2.0.1`
import $ivy.`edu.holycross.shot::histoutils:2.2.0`
import $ivy.`org.plotly-scala::plotly-almond:0.7.1`

Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/nomisma_2.12/2.0.1/nomisma_2.12-2.0.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/nomisma_2.12/2.0.1/nomisma_2.12-2.0.1.pom.sha1
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/nomisma_2.12/2.0.1/nomisma_2.12-2.0.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.1.1/xcite_2.12-4.1.1.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ohco2_2.12/10.16.0/ohco2_2.12-10.16.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/histoutils_2.12/2.2.0/histoutils_2.12-2.2.0.pom
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/ohco2_2.12/10.16.0/ohco2_2.12-10.16.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/histoutils_2.12/2.2.0/histoutils_2.12-2.2.0.pom.sha1
Downloaded https://repo1.maven.org/maven2/edu/holycross/shot/cite/xcite_2.12/4.1.1/xcite_2.12-4.1.1.pom.sha1
Downloaded https://repo1.maven.

Downloaded https://repo1.maven.org/maven2/org/scala-lang/scala-library/2.12.8/scala-library-2.12.8-sources.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/cex_2.12/6.3.3/cex_2.12-6.3.3.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/cex_2.12/6.3.3/cex_2.12-6.3.3-sources.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/cite/xcite_2.12/4.1.1/xcite_2.12-4.1.1-sources.jar
Downloaded https://repo1.maven.org/maven2/org/wvlet/airframe/airframe-log_2.12/19.8.10/airframe-log_2.12-19.8.10-sources.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/histoutils_2.12/2.2.0/histoutils_2.12-2.2.0-sources.jar
Downloaded https://dl.bintray.com/neelsmith/maven/edu/holycross/shot/seqcomp_2.12/1.0.0/seqcomp_2.12-1.0.0-sources.jar
Downloaded https://repo1.maven.org/maven2/org/scala-lang/modules/scala-collection-compat_2.12/2.1.1/scala-collection-compat_2.12-2.1.1.jar
Downloaded https://dl.bintray.com/neelsmith/mav

import $ivy.$                                  

import $ivy.$                                     

import $ivy.$                                      


## Load the CRRO data set

Load the CRRO data set from a `.cex` file in github repository.

In [3]:
import edu.holycross.shot.nomisma._
import edu.holycross.shot.histoutils._

import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
// Set display defaults suggested for use in Jupyter NBs:
repl.pprinter() = repl.pprinter().copy(defaultHeight = 3)

import edu.holycross.shot.nomisma._

import edu.holycross.shot.histoutils._


import plotly._, plotly.element._, plotly.layout._, plotly.Almond._
// Set display defaults suggested for use in Jupyter NBs:


In [4]:
val crroCex = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/crro-2019-12-03.cex"
val crro = CrroSource.fromUrl(crroCex)

// Check results:
require(crro.size > 2000)   

Dec 11, 2019 12:58:54 PM wvlet.log.Logger log
INFO: Reading 2281 lines of CEX data.
Dec 11, 2019 12:58:54 PM wvlet.log.Logger log
INFO: Created Crro with 2281 issues.


crroCex: String = "https://raw.githubusercontent.com/neelsmith/nomisma/master/cex/crro-2019-12-03.cex"
crro: Crro = Crro(
  Vector(
...

In [5]:
val obvLegends = crro.hasObvLegend.issues.map(_.obvLegend)
val revLegends = crro.hasRevLegend.issues.map(_.revLegend)
val allLegends = obvLegends ++ revLegends

val legendIssues = crro.hasObvLegend.issues ++ crro.hasRevLegend.issues
require (allLegends.size == legendIssues.size)

obvLegends: Vector[String] = Vector(
  "M\u00b7ANTON\u00b7IMP\u00b7III\u00b7VIR\u00b7R\u00b7P\u00b7C\u00b7AVG",
...
revLegends: Vector[String] = Vector(
  "CAESAR\u00b7IMP\u00b7PONT\u00b7III\u00b7VIR\u00b7R\u00b7P\u00b7C",
...
allLegends: Vector[String] = Vector(
  "M\u00b7ANTON\u00b7IMP\u00b7III\u00b7VIR\u00b7R\u00b7P\u00b7C\u00b7AVG",
...
legendIssues: Vector[CrroIssue] = Vector(
  CrroIssue(
...

In [6]:
println("Obv legends: " + obvLegends.size)
println("Rev legends: " + revLegends.size)
println("All: " + allLegends.size)

Obv legends: 1144
Rev legends: 2132
All: 3276


In [7]:
val allChars = allLegends.map(_.toVector).flatten
println("All chars: " + allChars.size)

All chars: 36208


allChars: Vector[Char] = Vector(
  'M',
...

In [8]:
val charFreqs = allChars.groupBy(c => c).map{ case (c,v)=> Frequency(c.toString,v.size) }
val charHist = edu.holycross.shot.histoutils.Histogram(charFreqs.toVector)

charFreqs: collection.immutable.Iterable[Frequency[String]] = List(
  Frequency("E", 1161),
...
charHist: edu.holycross.shot.histoutils.Histogram[String] = Histogram(
  Vector(
...

In [9]:
val charCounts = charHist.sorted.frequencies.map(_.count)
val charValues = charHist.sorted.frequencies.map(_.item)

charCounts: Vector[Int] = Vector(
  3725,
...
charValues: Vector[String] = Vector(
  "\u00b7",
...

In [10]:
val charHistPlot = Seq(
  Bar(x = charValues, y = charCounts)
)
plot(charHistPlot)

charHistPlot: Seq[Bar] = List(
  Bar(
...
res9_1: String = "plot-661a8473-3e51-4bd5-8088-b2d4d88ed78c"

In [11]:
val threshhold = 300
val rareChars = charHist.sorted.frequencies.filter(_.count < threshhold)
val lessRareChars = charHist.sorted.frequencies.filter(_.count >= threshhold)
println("Rare chars: \n" + rareChars.mkString("\n"))
println("\nCommon chars:\n" + lessRareChars.mkString("\n"))

Rare chars: 
Frequency(o,238)
Frequency(i,225)
Frequency(Q,225)
Frequency(t,224)
Frequency(s,216)
Frequency(e,212)
Frequency(?,210)
Frequency(n,201)
Frequency(r,191)
Frequency(a,177)
Frequency([,168)
Frequency(],167)
Frequency(d,146)
Frequency(c,107)
Frequency(H,98)
Frequency(.,84)
Frequency(),80)
Frequency((,80)
Frequency(/,75)
Frequency(b,45)
Frequency(l,44)
Frequency(h,34)
Frequency(g,31)
Frequency(w,27)
Frequency(p,23)
Frequency(v,21)
Frequency(f,19)
Frequency(",19)
Frequency(u,18)
Frequency(Y,16)
Frequency(K,16)
Frequency(;,14)
Frequency(m,11)
Frequency(k,10)
Frequency(:,8)
Frequency(|,7)
Frequency(Φ,3)
Frequency(',3)
Frequency(x,2)
Frequency(Ω,2)
Frequency(Α,1)
Frequency(Μ,1)
Frequency(Ν,1)
Frequency(Ι,1)
Frequency(-,1)
Frequency(Σ,1)
Frequency(Ρ,1)
Frequency(3,1)

Common chars:
Frequency(·,3725)
Frequency(I,3168)
Frequency(A,3119)
Frequency(R,2672)
Frequency( ,2290)
Frequency(M,2263)
Frequency(O,2104)
Frequency(S,1908)
Frequency(V,1868)
Frequency(C,1689)
Frequency(L,1442)
Freque

threshhold: Int = 300
rareChars: Vector[Frequency[String]] = Vector(
  Frequency("o", 238),
...
lessRareChars: Vector[Frequency[String]] = Vector(
  Frequency("\u00b7", 3725),
...

In [17]:
// Define some useful functions.


// True if String s composed only of allowable characters
def validOrtho(s: String) : Boolean = {
    val alphaChars = "ABCDEFGHIJKLMNOPQRSTUVWXYZ ·."
    val charChecks = for (c <- s.toVector) yield {
        alphaChars.contains(c)
    }
    val flatVals = charChecks.distinct
    (flatVals.size == 1) && (flatVals(0)== true)
}


// Map a Vector of OcreIssues to strings displaying ID and text of legends
def formatLegends(v: Vector[CrroIssue]) : Vector[String] = {
    v.map( issue => issue.id + " obv: " + issue.obvLegend + " rev: " + issue.revLegend)    
}


// Limit Vector of OcreIssues to those containing a specified string on either legend
def filterLegends(v : Vector[CrroIssue], subString : String) : Vector[CrroIssue] = {
    v.filter(issue => issue.obvLegend.contains(subString) || (issue.revLegend.contains(subString)))
}

defined function validOrtho
defined function formatLegends
defined function filterLegends

In [13]:
def pct(i1: Int, i2: Int): Float = {
    i1 * 100.0f / i2
}

defined function pct

In [14]:
val sheep = allLegends.filter(validOrtho(_))
val goats = allLegends.filterNot(validOrtho(_))
val total =  (sheep.size + goats.size)
val pct1 = pct(sheep.size, total)
val pct2 = pct(goats.size, total)

println("Sheep: " + sheep.size + " (" + pct1 + "%)")
println("Goats: " + goats.size  + " (" + pct2 + "%)")
println("Total:  " + total)
require(allLegends.size == total)



Sheep: 2720 (83.02808%)
Goats: 556 (16.971916%)
Total:  3276


sheep: Vector[String] = Vector(
  "M\u00b7ANTON\u00b7IMP\u00b7III\u00b7VIR\u00b7R\u00b7P\u00b7C\u00b7AVG",
...
goats: Vector[String] = Vector(
  "Inscription Obverse: signature P ANTON\u00b7AVG\u00b7IMP\u00b7III\u00b7COS\u0...
total: Int = 3276
pct1: Float = 83.02808F
pct2: Float = 16.971916F

In [16]:
val os = filterLegends(legendIssues, "o")
println("English 'o's: " + os.size)

val bigQs = filterLegends(legendIssues, "Q")
println("Latin 'Q's: " + bigQs.size)
//println(formatLegends(bigQs.take(10)).mkString("\n\n"))

val bigHs = filterLegends(legendIssues, "H")
println("Latin 'H's: " + bigHs.size)
//println(formatLegends(bigHs.take(10)).mkString("\n\n"))


val bigKs = filterLegends(legendIssues, "K")
println("Latin 'K's: " + bigKs.size)
///println(formatLegends(bigKs.take(10)).mkString("\n\n"))

val preciousDots = filterLegends(legendIssues, ".")
println("Precious dots: " +  preciousDots.size)
println(formatLegends(preciousDots.take(10)).mkString("\n\n"))


English 'o's: 287
Latin 'Q's: 344
Latin 'H's: 149
Latin 'K's: 27
Precious dots: 93
http://numismatics.org/crro/id/rrc-536.4 obv: ANT·AVGR·III·VIR·R·P·C rev: IMP. TER

http://numismatics.org/crro/id/rrc-138.1 obv: X rev: ROMA P. MAE

http://numismatics.org/crro/id/rrc-148.2 obv: S rev: ROMA Latin Q.MARI and S

http://numismatics.org/crro/id/rrc-149.2a obv: S rev: ROMA Latin L.MAMILI

http://numismatics.org/crro/id/rrc-149.2b obv: S rev: ROMA Latin L.MAMILI S

http://numismatics.org/crro/id/rrc-152.1a obv: X rev: ROMA SX.Q

http://numismatics.org/crro/id/rrc-152.1b obv: X rev: ROMA SX.Q

http://numismatics.org/crro/id/rrc-152.1c obv: X rev: ROMA SX.Q

http://numismatics.org/crro/id/rrc-153.1 obv: X rev: ROMA CN.CALP

http://numismatics.org/crro/id/rrc-154.1 obv: X rev: ROMA L.COIL


os: Vector[CrroIssue] = Vector(
  CrroIssue(
...
bigQs: Vector[CrroIssue] = Vector(
  CrroIssue(
...
bigHs: Vector[CrroIssue] = Vector(
  CrroIssue(
...
bigKs: Vector[CrroIssue] = Vector(
  CrroIssue(
...
preciousDots: Vector[CrroIssue] = Vector(
  CrroIssue(
...